In [29]:
import pandas as pd
import json

In [30]:
# Load the dataset
file_path = 'TRADES_CopyTr_90D_ROI.csv'
df = pd.read_csv(file_path)

# Drop rows with missing Trade_History for this analysis
df_clean = df.dropna(subset=['Trade_History']).copy()

# Inspect a few raw entries to identify formatting issues
raw_entries = df_clean['Trade_History'].head(5)

# Print out raw entries for inspection
for index, entry in enumerate(raw_entries):
    print(f"Entry {index}:\n{entry}\n")

Entry 0:
[{'time': 1718899656000, 'symbol': 'SOLUSDT', 'side': 'BUY', 'price': 132.537, 'fee': -0.9940275, 'feeAsset': 'USDT', 'quantity': 1988.055, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'SOL', 'qty': 15.0, 'positionSide': 'LONG', 'activeBuy': True}, {'time': 1718899618000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.27979617, 'feeAsset': 'USDT', 'quantity': 1398.98088, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 11484.0, 'positionSide': 'LONG', 'activeBuy': False}, {'time': 1718899618000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.03949404, 'feeAsset': 'USDT', 'quantity': 197.47022, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 1621.0, 'positionSide': 'LONG', 'activeBuy': False}, {'time': 1718899616000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.

In [31]:
def parse_json(trade_history):
    try:
        # Fix JSON formatting issues
        trade_history = trade_history.replace("'", '"')  # Replace single quotes with double quotes
        trade_history = trade_history.replace("None", "null")  # Replace Python None with JSON null
        trade_history = trade_history.replace("True", "true").replace("False", "false")  # Replace Python booleans
        # Ensure there are no trailing commas
        trade_history = trade_history.replace(",}", "}").replace(",]", "]")

        # Parse the JSON string
        return json.loads(trade_history)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e} in entry: {trade_history[:100]}...")
        return None

In [32]:

# Apply parsing to each entry
df_clean['Parsed_Trade_History'] = df_clean['Trade_History'].apply(parse_json)

In [33]:
# Check for any entries that couldn't be parsed
unparsed_entries = df_clean[df_clean['Parsed_Trade_History'].isnull()]

In [34]:
# Display the number of unparsed entries
num_unparsed_entries = len(unparsed_entries)

In [35]:
if not df_clean['Parsed_Trade_History'].dropna().empty:
    sample_trade_history = df_clean['Parsed_Trade_History'].dropna().iloc[0]
else:
    sample_trade_history = "No entries were successfully parsed."

num_unparsed_entries, sample_trade_history

(0,
 [{'time': 1718899656000,
   'symbol': 'SOLUSDT',
   'side': 'BUY',
   'price': 132.537,
   'fee': -0.9940275,
   'feeAsset': 'USDT',
   'quantity': 1988.055,
   'quantityAsset': 'USDT',
   'realizedProfit': 0.0,
   'realizedProfitAsset': 'USDT',
   'baseAsset': 'SOL',
   'qty': 15.0,
   'positionSide': 'LONG',
   'activeBuy': True},
  {'time': 1718899618000,
   'symbol': 'DOGEUSDT',
   'side': 'BUY',
   'price': 0.12182,
   'fee': -0.27979617,
   'feeAsset': 'USDT',
   'quantity': 1398.98088,
   'quantityAsset': 'USDT',
   'realizedProfit': 0.0,
   'realizedProfitAsset': 'USDT',
   'baseAsset': 'DOGE',
   'qty': 11484.0,
   'positionSide': 'LONG',
   'activeBuy': False},
  {'time': 1718899618000,
   'symbol': 'DOGEUSDT',
   'side': 'BUY',
   'price': 0.12182,
   'fee': -0.03949404,
   'feeAsset': 'USDT',
   'quantity': 197.47022,
   'quantityAsset': 'USDT',
   'realizedProfit': 0.0,
   'realizedProfitAsset': 'USDT',
   'baseAsset': 'DOGE',
   'qty': 1621.0,
   'positionSide': 'LON

In [36]:
import numpy as np

def calculate_metrics(trade_history):
    # Initialize metrics
    total_pnl = 0
    total_investment = 0
    wins = 0
    total_trades = len(trade_history)
    returns = []  # To calculate Sharpe Ratio
    equity_curve = []  # To calculate Maximum Drawdown

    # Iterate through trades
    for trade in trade_history:
        pnl = trade.get('realizedProfit', 0)
        price = trade.get('price', 0)
        qty = trade.get('qty', 0)

        # Calculate PnL and total investment
        total_pnl += pnl
        investment = price * qty
        total_investment += investment

        # Calculate return for Sharpe Ratio
        if investment > 0:
            trade_return = pnl / investment
            returns.append(trade_return)

        # Track equity curve for Maximum Drawdown
        if equity_curve:
            equity_curve.append(equity_curve[-1] + pnl)
        else:
            equity_curve.append(pnl)

        # Count wins
        if pnl > 0:
            wins += 1

    # Calculate ROI
    roi = (total_pnl / total_investment) * 100 if total_investment != 0 else 0

    # Calculate Win Rate
    win_rate = (wins / total_trades) * 100 if total_trades != 0 else 0

    # Calculate Sharpe Ratio (using mean return and standard deviation of returns)
    mean_return = np.mean(returns) if returns else 0
    std_dev_return = np.std(returns) if returns else 1
    sharpe_ratio = (mean_return / std_dev_return) if std_dev_return != 0 else 0

    # Calculate Maximum Drawdown (MDD)
    peak = equity_curve[0] if equity_curve else 0
    max_drawdown = 0
    for equity in equity_curve:
        if equity > peak:
            peak = equity
        drawdown = (peak - equity) / peak if peak != 0 else 0
        if drawdown > max_drawdown:
            max_drawdown = drawdown

    return {
        'ROI': roi,
        'PnL': total_pnl,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown * 100,
        'Win Rate': win_rate,
        'Win Positions': wins,
        'Total Positions': total_trades
    }

# Apply metrics calculation to each account
df_clean['Metrics'] = df_clean['Parsed_Trade_History'].apply(calculate_metrics)

# Display calculated metrics for the first account
df_clean[['Port_IDs', 'Metrics']].head()


,Port_IDs,Metrics
0,3925368433214965504,"{'ROI': 0.37530306371635236, 'PnL': 6789.43673..."
1,4002413037164645377,"{'ROI': 2.492177630013404, 'PnL': 3686.9697606..."
2,3923766029921022977,"{'ROI': 0.15648862124032617, 'PnL': 772.729031..."
3,3994879592543698688,"{'ROI': 0.8283465412325384, 'PnL': 3658.765062..."
4,3926423286576838657,"{'ROI': 0.13545884037560071, 'PnL': 1309.45995..."


In [37]:
# Define weights for each metric
weights = {
    'ROI': 0.30,
    'PnL': 0.25,
    'Sharpe Ratio': 0.20,
    'Max Drawdown': -0.15,  # Negative weight since lower drawdown is better
    'Win Rate': 0.10
}

# Function to calculate a composite score for each account
def calculate_score(metrics):
    score = (
        weights['ROI'] * metrics['ROI'] +
        weights['PnL'] * metrics['PnL'] +
        weights['Sharpe Ratio'] * metrics['Sharpe Ratio'] +
        weights['Max Drawdown'] * (100 - metrics['Max Drawdown']) +  # Invert drawdown
        weights['Win Rate'] * metrics['Win Rate']
    )
    return score

# Apply the score calculation
df_clean['Score'] = df_clean['Metrics'].apply(calculate_score)

# Rank the accounts by score
df_clean['Rank'] = df_clean['Score'].rank(ascending=False)

# Sort by rank and get the top 20 accounts
top_20_accounts = df_clean.sort_values(by='Score', ascending=False).head(20)

# Display the top 20 accounts
top_20_accounts[['Port_IDs', 'Metrics', 'Score', 'Rank']]


,Port_IDs,Metrics,Score,Rank
82,4020204877254599680,"{'ROI': 0.3115608194225942, 'PnL': 71998.85595...",18108.509209,1.0
127,3999240873283311617,"{'ROI': 0.5436734530219961, 'PnL': 42574.47367...",10634.924284,2.0
23,4021669203289716224,"{'ROI': 0.4044195741684927, 'PnL': 26427.33159...",6597.525021,3.0
64,3960874214179953664,"{'ROI': 0.23329625287420774, 'PnL': 19567.4712...",5400.161150,4.0
119,3907081197088384000,"{'ROI': 1.0992418198530567, 'PnL': 18015.99737...",4499.269374,5.0
147,4028701921959171840,"{'ROI': 0.40648048877447035, 'PnL': 17601.4013...",4391.714378,6.0
22,3956076827719377409,"{'ROI': 1.032229831016538, 'PnL': 16790.012238...",4191.796735,7.0
35,3986814617275053313,"{'ROI': 0.8716283426330235, 'PnL': 16337.46188...",4077.709111,8.0
68,3966142151544441601,"{'ROI': 0.30386397204483057, 'PnL': 4883.80077...",3932.127690,9.0
87,4022565861939831809,"{'ROI': 0.2330795004708387, 'PnL': 14197.57707...",3541.494675,10.0


In [43]:
# Export metrics and scores to a CSV file
# Update this path to a valid directory on your machine
output_path = 'D:/Trade_data_analysis/top_accounts_metrics.csv'

# Save the DataFrame to the CSV file
df_clean.to_csv(output_path, index=False)

# Display the top 20 accounts
top_20_accounts_display = top_20_accounts[['Port_IDs', 'Metrics', 'Score', 'Rank']]
top_20_accounts_display


,Port_IDs,Metrics,Score,Rank
82,4020204877254599680,"{'ROI': 0.3115608194225942, 'PnL': 71998.85595...",18108.509209,1.0
127,3999240873283311617,"{'ROI': 0.5436734530219961, 'PnL': 42574.47367...",10634.924284,2.0
23,4021669203289716224,"{'ROI': 0.4044195741684927, 'PnL': 26427.33159...",6597.525021,3.0
64,3960874214179953664,"{'ROI': 0.23329625287420774, 'PnL': 19567.4712...",5400.161150,4.0
119,3907081197088384000,"{'ROI': 1.0992418198530567, 'PnL': 18015.99737...",4499.269374,5.0
147,4028701921959171840,"{'ROI': 0.40648048877447035, 'PnL': 17601.4013...",4391.714378,6.0
22,3956076827719377409,"{'ROI': 1.032229831016538, 'PnL': 16790.012238...",4191.796735,7.0
35,3986814617275053313,"{'ROI': 0.8716283426330235, 'PnL': 16337.46188...",4077.709111,8.0
68,3966142151544441601,"{'ROI': 0.30386397204483057, 'PnL': 4883.80077...",3932.127690,9.0
87,4022565861939831809,"{'ROI': 0.2330795004708387, 'PnL': 14197.57707...",3541.494675,10.0


In [44]:
# Define the output path for the top 20 accounts
top_20_output_path = 'D:/Trade_data_analysis/top_20_accounts.csv'

# Save the top 20 accounts to a CSV file
top_20_accounts.to_csv(top_20_output_path, index=False)

print(f"Top 20 accounts have been saved to {top_20_output_path}.")


Top 20 accounts have been saved to D:/Trade_data_analysis/top_20_accounts.csv.
